In [4]:
import gymnasium as gym
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm import tqdm
import torch.nn as nn
from itertools import count
import imageio


import torch as T
from torch import optim
import torch.nn.functional as F
from collections import deque , namedtuple
from itertools import count
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pygame 2.5.0 (SDL 2.28.0, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Planning:

In this notebook I am building an implementation of the PPO algorithm.

I believe that this is quite similar to the REINFORCE algorithm, so the first thing I want to do is understand the key differences so that I can quickly put a working version of the algorithm together to solve Lunar Lander.


TODO:
- [*] Build basic version of the pieces of PPO.
- [*] Implement the basic FCN that represents our policy
- [*] Create the Agent class
- [] Collect the trajectories of the agent through each episode
- [] Add policy updates to the agent class
- [] Add training loop and test ability to learn
- [] Implement minibatch updates
- [] Implement multiple epochs over the same data
- [] Implement concurrent training of the value function




In [5]:
# print gym environment information
env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5
    # render_mode="rgb_array"
)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation space: Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)
action space: Discrete(4)


In [6]:
class FCN(nn.Module):
    def __init__(self, n_observations, n_actions, fc1_dims=256, fc2_dims=256):
        super(FCN, self).__init__()
        self.layer1 = nn.Linear(n_observations, fc1_dims)
        self.layer2 = nn.Linear(fc1_dims, fc2_dims)
        self.layer3 = nn.Linear(fc2_dims, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        logits = self.layer3(x)
        return F.softmax(logits, dim=-1)


# Produces distribution of actions

In [10]:
import torch
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PPOAgent:
    def __init__(self, n_observations, n_actions, env, gamma=0.99, lr=0.001, clip_epsilon=0.2, gae_lambda =0.95 ):
        self.policy = FCN(n_observations, n_actions).to(device)
        self.value_network = FCN(n_observations, 1).to(device)
        self.policy_optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.value_optimizer = optim.Adam(self.value_network.parameters(), lr=lr)
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.clip_epsilon = clip_epsilon
        self.env = env

    def choose_act(self, state):
        """This function samples the distribution of actions from the policy.
        It then returns a distinct action from the distribution"""
        state_tensor = torch.tensor(state, dtype=torch.float32).to(device)
        state_tensor = state_tensor.unsqueeze(0)

        distribution = self.policy(state_tensor)
        action_selection = torch.multinomial(distribution, num_samples=1)
        return action_selection
    
    def collect_trajectories(self, num_steps= 2000):
        
        rewards, actions, distributions, next_states, dones, states = [], [], [], [], [], []

        state, info = env.reset()

        for _ in range(num_steps):
            action, distribution = self.choose_act(state)
            next_state, reward, done, _, _ = env.step(action)
            rewards.append(reward)
            actions.append(action)
            distributions.append(distribution)
            next_states.append(next_state)
            dones.append(done)
            states.append(state)

            if done:
                state, info = env.reset()
                
            else:
                state = next_state
        
            
        states = torch.tensor(states).float().to(device)
        actions = torch.tensor(actions).to(device)
        rewards = torch.tensor(rewards).float().to(device)
        next_states = torch.tensor(next_states).float().to(device)
        dones = torch.tensor(dones).float().to(device)
        distributions = torch.tensor(distributions).float().to(device)

        return states, actions, rewards, next_states, dones, distributions
    
    def calculate_advantanges(self, rewards, states, next_states, dones):
        
        values = self.value_network(states).detach().squeeze()
        next_values = self.value_network(next_states).detach().squeeze()

        advantages = torch.zeros_like(rewards)
        gae = torch.tensor(0.0).to(device)

        returns = torch.zeros_like(rewards)
        future_return = torch.tensor(0.0).to(device)
        
        for t in reversed(range(len(rewards) -1)):
            delta = rewards[t] + self.gamma * next_values[t] * (1 -dones[t]) - values[t]
            gae = delta + self.gamma * self.gae_lambda * (1-dones[t]) * gae
            advantages[t] = gae

            future_return = rewards[t] + self.gamma * (1 - dones[t]) * future_return
            returns[t] = future_return
        
        return advantages, returns
    
    def update_policy(self, returns, advantages):


    def train(self, epochs, mini_batch_size):        
        # here we will need to calculate advantage for our particular run, as well as set up our training loop.
        # For each episode we want it to collect trajectories, and then calculate advantage for that particular trajectory.
        # then once we have collected the trajectory we'll want to apply it using backprop and autograd.
        for i in range(epochs):
            states, actions, rewards, next_states, dones, distributions = self.collect_trajectories(num_episodes = 2000)
            advantages, returns = self.calculate_advantanges(rewards, states, next_states, dones)


            dataset = list(zip(states, actions,advantages, returns))
            np.random.shuffle(dataset)

            for batch_num in range(0, len(dataset), mini_batch_size):
                mini_batch = dataset[batch_num:batch_num + mini_batch_size]
                states_batch, actions_batch, advantages_batch, returns_batch = zip(*mini_batch)
                

                
                


            

            



In [52]:
o = env.observation_space.shape[0]
actions = env.action_space.n
Agey = PPOAgent(o,actions)
state, info = env.reset()
state_tensor = torch.tensor(state, dtype=torch.float32).to(device)

action = Agey.choose_act(state)
env.step(int(action))

(array([ 3.6557198e-03,  1.3889743e+00,  1.7849378e-01, -5.0053096e-01,
        -2.1347743e-03,  5.1997055e-04,  0.0000000e+00,  0.0000000e+00],
       dtype=float32),
 -0.9181189062574606,
 False,
 False,
 {})

In [48]:
def test_environment(env):
    state, _ = env.reset()
    action = env.action_space.sample()
    next_state, reward, done,truncated, _ = env.step(action)
    print(type(state))
    assert isinstance(state, np.ndarray), "State is not a numpy array"
    assert isinstance(reward, (int, float)), "Reward is not a scalar"
    assert isinstance(done, bool), "Done is not a boolean"

    print("Environment test passed!")

def test_act(agent, env):
    state, _ = env.reset()
    action = agent.act(state)
    assert 0 <= action <= env.action_space.n, "Action not in action space"

    print("`act` method test passed!")

def test_value_network_output(agent, env):
    state, _ = env.reset()
    value = agent.value_network(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0))

    assert value.size(0) == 1, "Value network output isn't a scalar for a single state"

    print("Value network output test passed!")


def test_advantage_calculation(agent, env):
    state, _ = env.reset()
    rewards, values = [], []

    for _ in range(10):
        action = agent.act(state)
        state, reward, done, _ = env.step(action)
        value = agent.value_network(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0))
        rewards.append(reward)
        values.append(value)

    advantages = agent.estimate_advantage(rewards, values)  # replace with your method name

    assert len(advantages) == 10, "Advantage calculation length mismatch"

    print("Advantage calculation test passed!")

def test_policy_update(agent):
    initial_weights = torch.cat([p.flatten() for p in agent.policy.parameters()])
    
    # Collect some data here (states, actions, rewards, ...)
    # and then update the policy
    agent.policy_update()  # replace with your method name

    updated_weights = torch.cat([p.flatten() for p in agent.policy.parameters()])
    assert not torch.equal(initial_weights, updated_weights), "Policy weights did not change after update"

    print("Policy update test passed!")

def test_policy_gradient(agent, env):
    state,_ = env.reset()
    action_probabilities = agent.policy(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0))
    chosen_action_prob = action_probabilities[0, env.action_space.sample()]  # Sample an action to mimic choosing
    chosen_action_prob.backward()
    for param in agent.policy.parameters():
        assert param.grad is not None, f"Gradient not set for {param}"
    print("Policy gradient test passed!")

def test_value_gradient(agent, env):
    state, _ = env.reset()
    value_estimate = agent.value_network(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0))
    value_loss = (value_estimate - torch.tensor([1.0], device=device))**2  # Dummy target value
    value_loss.backward()
    for param in agent.value_network.parameters():
        assert param.grad is not None, f"Gradient not set for {param}"
    print("Value gradient test passed!")

def test_gae_calculation(agent, env):
    states, rewards, dones, values = [], [], [], []
    state, info = env.reset()
    for _ in range(10):
        values.append(agent.value_network(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0)).item())
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
        states.append(state)
        rewards.append(reward)
        dones.append(done)

    advantages, _ = agent.compute_gae(rewards, dones, values, values)  # Assuming you have this method or similar

    assert len(advantages) == 10, "GAE calculation length mismatch"

    print("GAE calculation test passed!")

def test_clipped_objective(agent, env):
    old_probs, new_probs, advantages = [], [], []
    state, _ = env.reset()
    for _ in range(10):
        action_probabilities = agent.policy(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0))
        action = agent.act(state)
        old_probs.append(action_probabilities[0, action].item())
        state, _, _, _, _ = env.step(action)
        action_probabilities = agent.policy(torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0))
        new_probs.append(action_probabilities[0, action].item())
        advantages.append(1.0)  # dummy advantage

    surrogate_objective = agent.compute_surrogate_objective(old_probs, new_probs, advantages)  # You might have a different method name

    assert surrogate_objective <= (1 + agent.clip_epsilon) * advantages[0], "Objective exceeds upper bound"
    assert surrogate_objective >= (1 - agent.clip_epsilon) * advantages[0], "Objective falls below lower bound"

    print("Clipped objective test passed!")


In [50]:
o = env.observation_space.shape[0]
actions = env.action_space.n
Agey = PPOAgent(o,actions)
test_environment(env)
test_act(Agey, env)
test_value_network_output(agent=Agey, env=env)
test_advantage_calculation(agent=Agey, env=env)
test_policy_update(agent=Agey)
test_policy_gradient(agent=Agey, env=env)
test_value_gradient(agent=Agey, env=env)
test_gae_calculation(agent=Agey, env=env)
test_clipped_objective(agent=Agey, env=env)

<class 'numpy.ndarray'>
Environment test passed!
`act` method test passed!
Value network output test passed!


AssertionError: tensor([[2]]) (<class 'torch.Tensor'>) invalid 